# FAQ Scraper API Walkthrough

### Setup

This notebook assumes you have portforwarding set up on your local system and user access account has forward kube permissions. To setup portforwarding, clone this repo https://github.com/chatatechnologies/devops to your local system. 

1) In a terminal navigate to the local directory containing the cloned devops repo

2) run `forward_kube`

3) Select option `4`, cass-staging

A list of all kube services will scroll. You should see `faq-parser-app avaiable on localhost 8115`.
Keep this terminal open while interacting with the API. 

### Test Run

In [4]:
import requests
import json

In [5]:
BASE = 'http://127.0.0.1:8115/'             #Localhost:forwarded_port
url = 'https://chata.ai/autoql/faq/'        #url to Scrape
request_data = {'url': url}

In [ ]:
response = requests.put(BASE + 'faq_parser', request_data)
print(response.json())

### Saving To GCP Bucket

To save this FAQ (and its associated metadata) to a subdirectory inside nlp_resources:

In [6]:

# Little Function to help with readability
import pprint
from colorama import Fore
from colorama import Style

def fabulous(response_json):
    response_json = json.loads(response_json)
    if 'status' in response_json.keys():
        print(f"{Fore.RED}Status: {response_json['status']}{Style.RESET_ALL}", end='\n')
    if 'data' in response_json.keys():
        if response_json['data']:
            for qa in response_json['data']:
                print(f'{Fore.GREEN}[{int(qa)+1}]{Style.RESET_ALL}')
                print(f'{Fore.MAGENTA}Question:{Style.RESET_ALL}')
                pprint.pprint(f'{response_json["data"][qa]["question"]}')
                print(f'{Fore.CYAN}Answer:{Style.RESET_ALL}')
                pprint.pprint(f'{response_json["data"][qa]["answer"]}')
        if 'save_dir' in response_json.keys():
            if response_json['save_dir']:
                print(f"{Fore.RED}Save Location: {response_json['save_dir']}{Style.RESET_ALL}", end='\n')
    else:
        print(f'{Fore.RED}Parser found nothing to parse{Style.RESET_ALL}')

In [7]:
BASE = 'http://127.0.0.1:8115/'   
url = 'https://www.tamu.edu/about/faq.html'
bucket_dir = 'knowledge/test'      #this will save the data to nlp_resources/knowledge/test
request_data = {'url': url, 'save':True, 'bucket_dir': bucket_dir }

In [ ]:
response = requests.put(BASE + 'faq_parser', request_data)
fabulous(response.json())